In [ ]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from PIL import Image
import numpy as np

import import_ipynb
from bg_remover_u2net import load_u2net, predict_mask

# check for gpu availability
print("Has GPU:", torch.cuda.is_available())

In [ ]:
device = 'cpu' # cuda, if gpu available
u2net_path = '../u2net.pth'

load stable diffusion model

In [ ]:
def load_sd_controlnet(device):
    pipe_model = "runwayml/stable-diffusion-v1-5"
    controlnet_model = "lllyasviel/sd-controlnet-seg"

    controlnet = ControlNetModel.from_pretrained(controlnet_model, torch_dtype=torch.float32)

    pipe = StableDiffusionControlNetPipeline.from_pretrained(pipe_model, controlnet=controlnet, torch_dtype=torch.float32)
    
    pipe.enable_attention_slicing()
    pipe.to(device)

    return pipe

generate background image

In [ ]:
def generate_background(prompt, image_path, pipe, u2net, guide, steps):
    image = Image.open(image_path)
    w, h = image.size

    subject_mask, _ = predict_mask(image_path, u2net)
    subject_mask = Image.fromarray(((1-subject_mask) * 255).astype(np.uint8))
    subject_mask = subject_mask.convert("RGB") 

    result = pipe(
        prompt = prompt,
        image = subject_mask,
        width = w,
        height = h,
        guidance_scale = guide,
        num_inference_steps = steps
    )

    return result.images[0]

testing the script

In [ ]:
if __name__ == '__main__':
    
    pipe = load_sd_controlnet(device)
    u2net = load_u2net(u2net_path)

In [ ]:
if __name__ == '__main__':

    image_path = '../images/img1.jpg'
    prompt = "living room with blue wallpaper on the wall and sunlight"
    
    background = generate_background(prompt, image_path, pipe, u2net, 7, 20)
    background.show()